# NSL export to CoLDP: lichens

In [34]:
import pandas as pd
import numpy as np

namefile = 'data/lichens/LNI-names-2023-06-24-0038.csv'
taxonfile = 'data/lichens/ABRSL-taxon-2023-06-24-0056.csv'

### Names

In [35]:
df_names = pd.read_csv(namefile)

# Remove excess columns
columns = [
    'scientificNameID', 
    'nameAccordingToID', 
    'taxonRank', 
    'scientificName', 
    'genericName', 
    'specificEpithet', 
    'infraspecificEpithet', 
    'scientificNameAuthorship', 
    'nomenclaturalStatus',
    'originalNameUsageID', 
    'originalNameUsage'
]

df1 = df_names[columns]

# Instance ID is in nameAccordingToID for reasons unfathomable; rename to taxonID
df1 = df1.rename(columns={'nameAccordingToID': 'taxonID'})

# Filter for names with originalNameUsage
# Replace instance IDs in originalNameUsageID with scientificNameIDs; we will not need taxonID anymore after that
df2 = df1[~df1['originalNameUsageID'].isna()].merge(df1, how='left', left_on='originalNameUsageID', right_on='taxonID')[[
    'scientificNameID_x', 
    'taxonRank_x', 
    'scientificName_x', 
    'genericName_x', 
    'specificEpithet_x', 
    'infraspecificEpithet_x', 
    'scientificNameAuthorship_x', 
    'nomenclaturalStatus_x', 
    'scientificNameID_y', 
    'scientificName_y'
]]

column_mappings = {
    'scientificNameID_x': 'scientificNameID',
    'scientificName_x': 'scientificName',
    'taxonRank_x': 'taxonRank',
    'genericName_x': 'genericName',
    'specificEpithet_x': 'specificEpithet',
    'infraspecificEpithet_x': 'infraspecificEpithet',
    'scientificNameAuthorship_x': 'scientificNameAuthorship',
    'nomenclaturalStatus_x': 'nomenclaturalStatus',
    'scientificNameID_y': 'originalNameUsageID',
    'scientificName_y': 'originalNameUsage'
}

df2 = df2.rename(columns=column_mappings)

# Filter for names without originalNameUsage
df3 = df_names[df_names['originalNameUsage'].isna()][columns]
df3.drop(columns=['nameAccordingToID'], inplace=True)

# Merge dataframes
df5 = pd.concat([df2, df3])
df5 = df5.sort_values(by='scientificName')

# Make taxonRank lowercase
df5['taxonRank'] = df5['taxonRank'].str.lower()

# Map Latin rank names to less wankerish English ones
ranks = {
    'genus': 'genus',
    'species': 'species',
    'familia': 'family',
    'subfamilia': 'subfamily',
    'ordo': 'order',
    'subordo': 'suborder',
    'superordo': 'superorder',
    'subspecies': 'subspecies',
    'classis': 'class',
    'subclassis': 'subclass',
    'subdivision': 'subphylum',
    'subbdivision': 'subphylum',
    'varietas': 'variety',
    'subvarietas': 'subvariety',
    'subgenus': 'subgenus',
    'superspecies': 'superspecies',
    'forma': 'form',
    'division': 'phylum',
    'regnum': 'kingdom',
    'special form': 'special form',
    'sectio': 'section',
    'regio': 'domain',
    '[unknown]': '[unknown]',
    '[unranked]': '[unranked]'
}

taxon_ranks = []
uninomials = []
generic_names = []
for index, row in df5.iterrows():
    taxon_ranks.append(ranks[row['taxonRank']])

    uninomial = np.NaN
    generic_name = row['genericName']

    if not isinstance(row['genericName'], str):
        if not isinstance(row['scientificNameAuthorship'], str):
            uninomial = row['scientificName']
        else:
            uninomial = row['scientificName'][0:len(row['scientificName'])-len(row['scientificNameAuthorship'])-1]

    if row['taxonRank'] == 'genus':
        uninomial = row['genericName']
        generic_name = np.NaN

    uninomials.append(uninomial)
    generic_names.append(generic_name)

df5['taxonRank'] = taxon_ranks
df5['genericName'] = generic_names
df5['uninomial'] = uninomials

# Add nomenclaturalCode; 'ICN' in GBIF vocab. but 'botanical' in CoLDP
df5['code'] = 'botanical'

df5 = df5[[
    'scientificNameID', 
    'taxonRank', 
    'scientificName', 
    'uninomial',
    'genericName', 
    'specificEpithet', 
    'infraspecificEpithet', 
    'scientificNameAuthorship', 
    'code', 
    'nomenclaturalStatus', 
    'originalNameUsageID', 
    'originalNameUsage'
  ]]

df_nam = df5
df5

,scientificNameID,taxonRank,scientificName,uninomial,genericName,specificEpithet,infraspecificEpithet,scientificNameAuthorship,code,nomenclaturalStatus,originalNameUsageID,originalNameUsage
2040,https://id.biodiversity.org.au/name/lichen/300...,genus,Aabaarnia Diederich,Aabaarnia,NaN,NaN,NaN,Diederich,botanical,NaN,NaN,NaN
2039,https://id.biodiversity.org.au/name/lichen/300...,species,Aabaarnia siphulicola Diederich,NaN,Aabaarnia,siphulicola,NaN,Diederich,botanical,NaN,NaN,NaN
2042,https://id.biodiversity.org.au/name/lichen/300...,genus,Abrothallus De Not.,Abrothallus,NaN,NaN,NaN,De Not.,botanical,NaN,NaN,NaN
2041,https://id.biodiversity.org.au/name/lichen/300...,species,Abrothallus parmeliarum (Sommerf.) Arnold,NaN,Abrothallus,parmeliarum,NaN,(Sommerf.) Arnold,botanical,NaN,NaN,NaN
7356,https://id.biodiversity.org.au/name/lichen/300...,genus,Absconditella Vězda,Absconditella,NaN,NaN,NaN,Vězda,botanical,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8262,https://id.biodiversity.org.au/name/lichen/300...,genus,Zwackhiomyces Grube & Hafellner,Zwackhiomyces,NaN,NaN,NaN,Grube & Hafellner,botanical,NaN,NaN,NaN
909,https://id.biodiversity.org.au/name/lichen/300...,species,Zwackhiomyces cladoniae (C.W.Dodge) Diederich,NaN,Zwackhiomyces,cladoniae,NaN,(C.W.Dodge) Diederich,botanical,NaN,https://id.biodiversity.org.au/name/lichen/300...,Didymella cladoniae C.W.Dodge
8259,https://id.biodiversity.org.au/name/lichen/300...,species,Zwackhiomyces kantvilasii S.Y.Kondr.,NaN,Zwackhiomyces,kantvilasii,NaN,S.Y.Kondr.,botanical,NaN,NaN,NaN
8260,https://id.biodiversity.org.au/name/lichen/300...,species,Zwackhiomyces lecanorae (Stein) Nik.Hoffm. & H...,NaN,Zwackhiomyces,lecanorae,NaN,(Stein) Nik.Hoffm. & Hafellner,botanical,NaN,NaN,NaN


### Name relations

In [36]:
df_name_relationships = df5[~df5['originalNameUsageID'].isna()][['scientificNameID', 'scientificName', 'originalNameUsageID', 'originalNameUsage']]

df_namerel = df_name_relationships[['scientificNameID', 'originalNameUsageID']]
df_namerel['type'] = 'basionym'
df_namerel.rename(columns={'scientificNameID': 'nameID', 'originalNameUsageID': 'relatedNameID'}, inplace=True)

# Remove originalNameUsage columns from Names
drop_columns = [
    'originalNameUsageID',
    'originalNameUsage'
]

rename_columns = {
    'scientificNameID': 'ID',
    'taxonRank': 'rank',
    'scientificName': 'scientificName',
    'genericName': 'genus',
    'scientificNameAuthorship': 'authorship',
    'nomenclaturalStatus': 'status'
}

df5.drop(columns=drop_columns, inplace=True)
df5.rename(columns=rename_columns, inplace=True)

/tmp/ipykernel_288785/3173867388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_namerel['type'] = 'basionym'
/tmp/ipykernel_288785/3173867388.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_namerel.rename(columns={'scientificNameID': 'nameID', 'originalNameUsageID': 'relatedNameID'}, inplace=True)


### Taxa

In [37]:
# Read Taxon export into dataframe
df_taxa = pd.read_csv(taxonfile)
df_taxa = df_taxa[df_taxa['nameType'] == 'scientific']

# Create dataframe with IDs
# This is used later to replace tree element IDs (in taxonID) with instance IDs (in taxonConceptID)
df_id = df_taxa[['taxonID', 'taxonConceptID']]

In [38]:
# Filter on accepted names; these are the taxa
df_tax = df_taxa[df_taxa['taxonomicStatus'] == 'accepted']

# Replace tree element IDs in taxonID with instance IDs (see above)
df_tax['taxonID'] = df_tax['taxonConceptID']

# Replace tree element IDs in parentNameUsageID with instance IDs
df_tax = df_tax.merge(df_id, how='left', left_on='parentNameUsageID', right_on='taxonID')
df_tax['parentNameUsageID'] = df_tax['taxonConceptID_y']
df_tax.drop(columns=['taxonID_y', 'taxonConceptID_y'], inplace=True)
df_tax.rename(columns={'taxonID_x': 'taxonID', 'taxonConceptID_x': 'taxonConceptID'}, inplace=True)

df_tax = df_tax[['taxonID',
 'scientificNameID',
 'scientificName',
 'scientificNameAuthorship',
 'nameAccordingTo',
 'nameAccordingToID',
 'parentNameUsageID',
 'taxonRank',
 'taxonRankSortOrder',
 'kingdom',
 'class',
 'subclass',
 'family',
 'taxonConceptID',
 'taxonRemarks',
 'higherClassification'
]]

# Translate ranks into English
df_tax['taxonRank'] = df_tax['taxonRank'].str.lower()
ranks = {
    'genus': 'genus',
    'species': 'species',
    'familia': 'family',
    'subfamilia': 'subfamily',
    'ordo': 'order',
    'subordo': 'suborder',
    'superordo': 'superorder',
    'subspecies': 'subspecies',
    'classis': 'class',
    'subclassis': 'subclass',
    'subdivision': 'subphylum',
    'subbdivision': 'subphylum',
    'varietas': 'variety',
    'subgenus': 'subgenus',
    'superspecies': 'superspecies',
    'forma': 'form',
    'division': 'phylum',
    'regnum': 'kingdom',
    'special form': 'special form',
    'sectio': 'section',
    'regio': 'domain',
    '[unknown]': '[unknown]'
}

taxon_ranks = []
for index, row in df_tax.iterrows():
    taxon_ranks.append(ranks[row['taxonRank']])

df_tax['taxonRank'] = taxon_ranks


/tmp/ipykernel_288785/2562605175.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tax['taxonID'] = df_tax['taxonConceptID']


#### Higher classification

In [39]:
# Create dictionary with higher taxa from instance ID
def get_higher_taxa(id, higher={}):
    row = df_tax.loc[df_tax['taxonID'] == id]

    if len(row) > 0:
        parent = row[[
            'taxonRank', 
            'scientificName', 
            'scientificNameAuthorship', 
            'parentNameUsageID']].to_dict(orient='records')[0]
        
        if isinstance(parent['scientificNameAuthorship'], str):
            higher[parent['taxonRank']] = parent['scientificName'][0:len(parent['scientificName'])-len(parent['scientificNameAuthorship'])-1]
        else:
            higher[parent['taxonRank']] = parent['scientificName']

        if isinstance(parent['parentNameUsageID'], str):
            return get_higher_taxa(parent['parentNameUsageID'], higher)
        else:
            return higher
    else:
        return higher

In [40]:
# Create higher classification
def create_higher_classification(id):
    higher = get_higher_taxa(id, {})
    keys = list(higher.keys())

    hcl = {}

    hcl['taxonID'] = id
    hcl['species'] = higher['species'] if 'species' in keys else np.NaN
    hcl['section'] = higher['section'] if 'section' in keys else np.NaN
    hcl['subgenus'] = higher['subgenus'] if 'subgenus' in keys else np.NaN
    hcl['genus'] = higher['genus'] if 'genus' in keys else np.NaN
    hcl['subtribe'] = higher['subtribe'] if 'subtribe' in keys else np.NaN
    hcl['tribe'] = higher['tribe'] if 'tribe' in keys else np.NaN
    hcl['subfamily'] = higher['subfamily'] if 'subfamily' in keys else np.NaN
    hcl['family'] = higher['family'] if 'family' in keys else np.NaN
    hcl['superfamily'] = higher['superfamily'] if 'superfamily' in keys else np.NaN
    hcl['suborder'] = higher['suborder'] if 'suborder' in keys else np.NaN
    hcl['order'] = higher['order'] if 'order' in keys else np.NaN
    hcl['subclass'] = higher['subclass'] if 'subclass' in keys else np.NaN
    hcl['class'] = higher['class'] if 'class' in keys else np.NaN
    hcl['subphylum'] = higher['subphylum'] if 'subphylum' in keys else np.NaN
    hcl['phylum'] = higher['phylum'] if 'phylum' in keys else np.NaN
    hcl['kingdom'] = higher['kingdom'] if 'kingdom' in keys else np.NaN

    cl = list(higher.values())
    for i in range(len(cl) // 2):
        cl[i], cl[-1 - i] = cl[-1 - i], cl[i]

    hcl['classification'] = ' | '.join(cl)

    return hcl

create_higher_classification('https://id.biodiversity.org.au/instance/lichen/30043034')


{'taxonID': 'https://id.biodiversity.org.au/instance/lichen/30043034',
 'species': 'Arthonia amoena',
 'section': nan,
 'subgenus': nan,
 'genus': 'Arthonia',
 'subtribe': nan,
 'tribe': nan,
 'subfamily': nan,
 'family': 'Arthoniaceae',
 'superfamily': nan,
 'suborder': nan,
 'order': 'Arthoniales',
 'subclass': nan,
 'class': 'Arthoniomycetes',
 'subphylum': 'Pezizomycotina',
 'phylum': 'Ascomycota',
 'kingdom': 'Fungi',
 'classification': 'Fungi | Ascomycota | Pezizomycotina | Arthoniomycetes | Arthoniales | Arthoniaceae | Arthonia | Arthonia amoena'}

In [41]:
cl = []
for index, row in df_tax.iterrows():
    cl.append(create_higher_classification(row['taxonID']))

df_higher = pd.DataFrame.from_dict(cl)

df_higher = df_higher.merge(df_tax[['taxonID', 'scientificName', 'taxonRank']], how='left', left_on='taxonID', right_on='taxonID')

df_higher = df_higher[['taxonID',
 'scientificName',
 'taxonRank',
 'kingdom',
 'phylum',
 'subphylum',
 'class',
 'subclass',
 'order',
 'suborder',
 'superfamily',
 'family',
 'tribe',
 'subfamily',
 'subtribe',
 'genus',
 'subgenus',
 'section',
 'species',
 'classification']]

df_higher.rename(columns={'classification': 'higherClassification'}, inplace=True)

df_higher

,taxonID,scientificName,taxonRank,kingdom,phylum,subphylum,class,subclass,order,suborder,superfamily,family,tribe,subfamily,subtribe,genus,subgenus,section,species,higherClassification
0,https://id.biodiversity.org.au/instance/lichen...,Fungi,kingdom,Fungi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fungi
1,https://id.biodiversity.org.au/instance/lichen...,Agaricaceae,family,Fungi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agaricaceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fungi | Agaricaceae
2,https://id.biodiversity.org.au/instance/lichen...,Agaricus L.,genus,Fungi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agaricaceae,NaN,NaN,NaN,Agaricus,NaN,NaN,NaN,Fungi | Agaricaceae | Agaricus
3,https://id.biodiversity.org.au/instance/lichen...,Ascomycota,phylum,Fungi,Ascomycota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fungi | Ascomycota
4,https://id.biodiversity.org.au/instance/lichen...,Ascomycetes,class,Fungi,Ascomycota,NaN,Ascomycetes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fungi | Ascomycota | Ascomycetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4911,https://id.biodiversity.org.au/instance/lichen...,Roselliniella cladoniae (Anzi) Matzer & Hafellner,species,Fungi,NaN,NaN,NaN,NaN,Sordariales,NaN,NaN,NaN,NaN,NaN,NaN,Roselliniella,NaN,NaN,Roselliniella cladoniae,Fungi | Sordariales | Roselliniella | Rosellin...
4912,https://id.biodiversity.org.au/instance/lichen...,Roselliniella epiphylla Matzer & Hafellner,species,Fungi,NaN,NaN,NaN,NaN,Sordariales,NaN,NaN,NaN,NaN,NaN,NaN,Roselliniella,NaN,NaN,Roselliniella epiphylla,Fungi | Sordariales | Roselliniella | Rosellin...
4913,https://id.biodiversity.org.au/instance/lichen...,Roselliniella heterodeae Matzer & Hafellner,species,Fungi,NaN,NaN,NaN,NaN,Sordariales,NaN,NaN,NaN,NaN,NaN,NaN,Roselliniella,NaN,NaN,Roselliniella heterodeae,Fungi | Sordariales | Roselliniella | Rosellin...
4914,https://id.biodiversity.org.au/instance/lichen...,Thelotremataceae,family,Fungi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thelotremataceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fungi | Thelotremataceae


#### Synonyms

In [42]:
# Filter on synonyms
df_syn = df_taxa[df_taxa['taxonomicStatus'].isin(['synonym', 'nomenclatural synonym', 'taxonomic synonym']) & ~df_taxa['acceptedNameUsageID'].isna()][[
    'taxonID',
    'scientificNameID',
    'scientificName',
    'acceptedNameUsageID',
    'acceptedNameUsage',
    'taxonomicStatus'
]]

# Replace tree element IDs in acceptedNameUsageID with instance IDs
df_syn = df_syn.merge(df_id, how='left', left_on='acceptedNameUsageID', right_on='taxonID')

df_syn.drop(columns=['acceptedNameUsageID', 'taxonID_y'], inplace=True)
df_syn.rename(columns={
    'taxonID_x': 'ID',
    'taxonConceptID': 'taxonID',
    'scientificNameID': 'nameID'    
}, inplace=True)
df_syn = df_syn[['ID', 'taxonID', 'nameID', 'scientificName', 'acceptedNameUsage', 'taxonomicStatus']]

# Get accepted names
df_dwc_syn = df_syn.merge(df_tax[['taxonID', 'scientificName']], left_on='taxonID', right_on='taxonID')

df_dwc_syn.drop(columns=['taxonID', 'nameID', 'acceptedNameUsage'], inplace=True)

df_dwc_syn.rename(columns={
    'ID': 'taxonID',
    'scientificName_x': 'scientificName',
    'scientificName_y': 'acceptedNameUsage'
}, inplace=True)

df_syn.drop(columns=['scientificName', 'acceptedNameUsage', 'taxonomicStatus'], inplace=True)

df_syn

,ID,taxonID,nameID
0,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
1,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
2,https://id.biodiversity.org.au/instance/lichen...,NaN,https://id.biodiversity.org.au/name/lichen/300...
3,https://id.biodiversity.org.au/instance/lichen...,NaN,https://id.biodiversity.org.au/name/lichen/300...
4,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
...,...,...,...
2860,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
2861,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
2862,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...
2863,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...


#### Remove excess columns

In [43]:
df_tax = df_tax[['taxonID',
 'scientificNameID',
 'nameAccordingToID',
 'parentNameUsageID',
 'taxonRemarks']]

df_tax.rename(columns={
    'taxonID': 'ID',
    'scientificNameID': 'nameID',
    'nameAccordingToID': 'accordingToID',
    'parentNameUsageID': 'parentID',
    'taxonRemarks': 'remarks'
}, inplace=True)

df_tax

,ID,nameID,accordingToID,parentID,remarks
0,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,NaN,NaN
1,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/301...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
2,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
3,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
4,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/301...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
...,...,...,...,...,...
4911,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
4912,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
4913,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/300...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN
4914,https://id.biodiversity.org.au/instance/lichen...,https://id.biodiversity.org.au/name/lichen/301...,https://id.biodiversity.org.au/reference/liche...,https://id.biodiversity.org.au/instance/lichen...,NaN


### Reference

In [44]:
df_ref = df_taxa[['nameAccordingToID', 'nameAccordingTo']]
df_ref.drop_duplicates(inplace=True)
df_ref.rename(columns={'nameAccordingToID': 'ID', 'nameAccordingTo': 'citation'}, inplace=True)
df_ref

/tmp/ipykernel_288785/3187620901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref.drop_duplicates(inplace=True)
/tmp/ipykernel_288785/3187620901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ref.rename(columns={'nameAccordingToID': 'ID', 'nameAccordingTo': 'citation'}, inplace=True)


,ID,citation
0,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2016), Checklist of the Li..."
4,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2016), Checklist of Austra..."
33,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2020), Checklist of the Li..."
48,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2018), Checklist of the Li..."
261,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2017), Checklist of the Li..."
336,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2017), Checklist of Austra..."
6622,https://id.biodiversity.org.au/reference/liche...,"P.McCarthy (Comp.) (2018), Checklist of Austra..."


### Create CoLDP

In [45]:
import os
from zipfile import ZipFile

os.chdir('/home/niels/code/jupyter-notebooks/nsl_export/coldp/lichens')

df_tax.to_csv('taxon.tsv', sep='\t', index=False)
df_syn.to_csv('synonym.tsv', sep='\t', index=False)
df_nam.to_csv('name.tsv', sep='\t', index=False)
df_name_relationships.to_csv('dwc_basionyms.tsv', sep='\t', index=False)
df_namerel.to_csv('namerelation.tsv', sep='\t', index=False)
df_higher.to_csv('dwc_higherclassification.tsv', sep='\t', index=False)
df_dwc_syn.to_csv('dwc_synonym.tsv', sep='\t', index=False)
df_ref.to_csv('reference.tsv', sep='\t', index='False')

with ZipFile('nsl_lichens_coldp.zip', 'w') as zipobj:
    zipobj.write('taxon.tsv')
    zipobj.write('name.tsv')
    zipobj.write('synonym.tsv')
    zipobj.write('namerelation.tsv')
    zipobj.write('reference.tsv')

os.chdir('/home/niels/code/jupyter-notebooks/nsl_export')

